# **Chapter 5: First-Class Functions** 

Treating a function like an object

In [1]:
def factorial(n):
    ''' returns n! '''
    return 1 if n < 2 else n * factorial(n - 1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

' returns n! '

In [4]:
type(factorial)

function

   **Example 5-2. Use function through a different name, and pass function as argument**

In [5]:
fact = factorial

In [6]:
fact

<function __main__.factorial(n)>

In [7]:
fact(5)

120

In [8]:
map(factorial, range(11))

In [9]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

   **Example 5-3. Sorting a list of words by length**

In [10]:
fruits = ['strawberry', 'banana', 'apple', 'cranberry', 'papaya']
sorted(fruits, key=len)

['apple', 'banana', 'papaya', 'cranberry', 'strawberry']

   **Example 5-4. Sorting a list of words by their reverse spelling**

In [12]:
def reverse(word):
    return word[::-1]
reverse('testing')

'gnitset'

In [13]:
sorted(fruits, key=reverse)

['banana', 'papaya', 'apple', 'cranberry', 'strawberry']

   **Example 5-5. Lists of factorials produced with map and filter compared to alternatives coded as list          comprehensions**

In [14]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [15]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [16]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [17]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

   **Example 5-6. Sum of integers up to 99 performed with reduce and sum**

In [19]:
from functools import reduce
from operator import add
reduce(add, range(100))

4950

In [21]:
sum(range(100))

4950

   **Example 5-7. Sorting a list of words by their reversed spelling using lambda**
  

In [22]:
sorted(fruits, key=lambda word: word[::-1])

['banana', 'papaya', 'apple', 'cranberry', 'strawberry']

    Lundh's lambda Refactoring Recipe:
    1. Write a comment explaining what the heck that lambda does.
    2. Study the comment for a while, and think of a name that captures the essence of that comment.
    3. Convert the lambda to a def statement, using that name.
    4. Remove the comment.

   **Example 5-8. bingocall.py: A BingoCage does one thing: picks items from a shuffled list**

In [25]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [26]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [27]:
bingo()

0

In [28]:
bingo()

1

In [29]:
bingo()

LookupError: pick from empty BingoCage

In [30]:
callable(bingo)

True

   **Example 5-9. Listing attributes of functions that don't exist in plain instances**

In [33]:
class C: pass
obj = C()
def func(): pass
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

   **Example 5-10. tag generates HTML; a keyword-only argument cls is used to pass "class" attributes as a        workaround because class is a keyword in Python**

In [17]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value
                            in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                        (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [58]:
tag('br')

'<br />'

In [59]:
tag('p', 'hello')

'<p>hello</p>'

In [60]:
tag('p', 'hello', 'world')

'<p>hello</p>\n<p>world</p>'

In [62]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [63]:
tag('p', 'hello', 'world', cls='sidebar')

'<p class="sidebar">hello</p>\n<p class="sidebar">world</p>'

In [64]:
tag(content='testing', name='img')

'<img content="testing" />'

In [65]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

   **Example 5-15. Function to shorten a string by clipping at a space near the desired length**

In [4]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if (space_before >= 0):
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: 
        end = len(text)
    return text[:end].rstrip()

In [7]:
clip.__defaults__

(80,)

In [8]:
clip.__code__

<code object clip at 0x7ff6df91d190, file "/tmp/ipykernel_14186/2629348048.py", line 1>

In [9]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [10]:
clip.__code__.co_argcount

2

In [12]:
from inspect import signature
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [13]:
str(sig)

'(text, max_len=80)'

In [14]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


   **Example 5-18. Binding the function signature from the tag function to a dict of arguments**

In [19]:
import inspect
sig = inspect.signature(tag)
my_tag = {'name':'img', 'title':'Sunset Boulevard', 'src':'sunset.jpg', 'cls':'framed'}

bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [20]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [21]:
del my_tag['name']
bound_args = sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

   **Example 5-19. Annotated clip function**

In [23]:
def clip(text:str, max_len: 'int > 0' = 80) -> str:
    """Return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if (space_before >= 0):
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: 
        end = len(text)
    return text[:end].rstrip()

In [24]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

   **Example 5-20. Extracting annotations from the function signature**

In [27]:
from inspect import signature
sig = signature(clip)
sig.return_annotation

str

In [29]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


   **Example 5-21. Factorial implemented with reduce and an anonymous function**

In [30]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

   **Example 5-22. Factorial implemented with reduce and operator.mul**

In [32]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

   **Example 5-25. Demo of methodcaller: second test shows the binding of extra arguments**

In [33]:
from operator import methodcaller
s = 'the time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [35]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'the-time-has-come'

   **Example 5-26. Using partial as a two-argument function where a one-argument function callable is required**

In [39]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [37]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]